# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Загрузим необходимые библиотеки 

In [1]:
import nltk
import os
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords as nltk_stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline

Загрузим необходимый датафрейм

In [2]:
pth = '/datasets/toxic_comments.csv'

if os.path.exists(pth):
    data = pd.read_csv(pth)
    data_prophet = pd.read_csv(pth)
else:
    print('Something is wrong')

Выведем первые 10 строчек датафрейма и общую информацию

In [3]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


В датафрейме присутствует столбец "Unnamed: 0" не несущий пользы для обучения модели, удалим его

In [5]:
data = data.drop('Unnamed: 0', axis=1)

Разобьем датафрейм на выборку с целевыми признаками и на выборку с таргетными признаками 

In [6]:
features = data['text']
target = data['toxic']

Изучим баланс классов

In [7]:
proc_one = target[target==1].count() / len(data) * 100
proc_ziro = target[target==0].count() / len(data) * 100
print(f'В выборке процент класса "1" составляет: {proc_one:.2f}%\n\
В выборке процент класса "0" составляет: {proc_ziro:.2f}%')

В выборке процент класса "1" составляет: 10.16%
В выборке процент класса "0" составляет: 89.84%


Создадим функцию для лемматизации и чистки текста от лишних символов 

In [8]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN)

In [10]:
def lem_func(data):
    lemmatizer = WordNetLemmatizer()
    lem_text = []
    for i in range(len(data)):
        lem = " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(data[i])])
        resub = re.sub(r'[^a-zA-Z]',' ', lem)
        resub_split = resub.split()
        res = " ".join(resub_split)
        lem_text.append(res)
    
    return lem_text

Применим созданную ранее функцию 

In [11]:
features_lem = lem_func(features)
features_lem = pd.Series(features_lem, index=features.index)

Разобьем выборки с целевыми и таргетными признаками на обучающую и тестовую выборку

In [12]:
features_train, features_test, target_train, target_test = train_test_split(features_lem, target, test_size=0.25)

Загрузим функцию для очистки от стоп-слов 

In [13]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Вычислим TF-IDF для корпуса текстов

In [14]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

Проверим размерность полученных выборок

In [15]:
tf_idf_train.shape, tf_idf_test.shape, target_train.shape, target_test.shape

((119469, 135548), (39823, 135548), (119469,), (39823,))

В данном разделе была произведена подготовка датафрейма для обучения моделей, а именно: Удалены не несущие пользы столбцы, произведена лемматизация, произведена очистка от лишних символов и стоп-слов, созданы выборки для обучения и теста.

## Обучение

Обучим логистическую регрессию и найдем F1-меру с помощью кросс-валидации 

In [16]:
param_lr = {'C':range(1, 16, 2)}
model_lr = LogisticRegression(solver='liblinear', random_state=12345, class_weight='balanced', max_iter=200)
gs_lr = GridSearchCV(estimator=model_lr, param_grid=param_lr, cv=3, scoring='f1')
gs_lr.fit(tf_idf_train, target_train)
gs_lr.best_score_

0.7590438049262794

Обучим дерево решений и найдем F1-меру с помощью кросс-валидации 

In [17]:
param_tree = {'max_depth':range(1, 16), 'min_samples_split':range(2,12,2), 'min_samples_leaf':range(1,11,2)}
model_tree = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
gs_tree = GridSearchCV(estimator=model_tree, param_grid=param_tree, cv=3, scoring='f1')
gs_tree.fit(tf_idf_train, target_train)
gs_tree.best_score_

0.5894772458450557

Обучим случайный лес и найдем F1-меру с помощью кросс-валидации 

In [18]:
param_rf = {'max_depth':range(1, 16),'n_estimators':range(2, 103, 10)}
model_rf = RandomForestClassifier(random_state=12345, class_weight='balanced')
gs_rf = GridSearchCV(estimator=model_rf, param_grid=param_rf, cv=3, scoring='f1')
gs_rf.fit(tf_idf_train, target_train)
gs_rf.best_score_

0.3956651362509355

Лучше всех себя показала модель логистической регрессии с параметром F1 - 0.766 на кросс-валидации, ее и возьмем для финального тестирования 

Проведем финальное тестирования на лучшей модели

In [19]:
model_lr = gs_lr.best_estimator_
model_lr.fit(tf_idf_train, target_train)
predict = model_lr.predict(tf_idf_test)
f1_score(target_test, predict)

0.7569167710349538

На финальном тестировании значение F1 - 0.757, порог для прохождения модели 0.75. Результат итогового тестирования - успешный

Оценим адекватность предсказаний лучшей модели с помощью dummyclassifier’a

In [20]:
dummy_model = DummyClassifier(strategy='constant', constant = 1)
dummy_model.fit(tf_idf_train, target_train)
dummy_predict = dummy_model.predict(tf_idf_test)
f1_score(target_test, dummy_predict)

0.18329873862366278

Результат F1 dammy-модели - 0.183, результат лучшей модели 0.757. Тест на адекватность предсказаний пройден успешно. 

В данном разделе было произведено обучение 3 различных моделей с помощью кросс-валидации, а именно: 
- Логистическая регрессия – Параметр F1 – 0.759
- Дерево решений – Параметр F1 – 0.589
- Случайный лес – Параметр F1 – 0.396

Лучше всего себя показала модели логистической регрессии, ее и взяли для финального тестирования. На финальном тестировании логистическая регрессия показала результат F1 – 0.757, что выше необходимого порога в 0.75. Также модель сравнили с dummy-моделью для определение адекватности предсказаний, модель логистической регрессии показала результат F1 лучше на 0.579. 


## Выводы

В разделе 'Подотовка' была произведена подготовка датафрейма для обучения моделей, а именно: Удалены не несущие пользы столбцы, произведена лемматизация, произведена очистка от лишних символов и стоп-слов, созданы выборки для обучения и теста.

В данном разделе было произведено обучение 3 различных моделей с помощью кросс-валидации, а именно: 
- Логистическая регрессия – Параметр F1 – 0.759
- Дерево решений – Параметр F1 – 0.589
- Случайный лес – Параметр F1 – 0.396

Лучше всего себя показала модели логистической регрессии, ее и взяли для финального тестирования. На финальном тестировании логистическая регрессия показала результат F1 – 0.757, что выше необходимого порога в 0.75. Также модель сравнили с dummy-моделью для определение адекватности предсказаний, модель логистической регрессии показала результат F1 лучше на 0.579. 
